In [ ]:
pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.1 MB/s eta 0:00:00


In [ ]:
pip install networkx

In [ ]:
pip install matplotlib

In [20]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from collections import Counter
import time

def Q1(strain_names, common_length):
    # Set your email address for Entrez (required by NCBI)
    Entrez.email = "shivam21489@iiitd.ac.in"  # Replace with your email address

    # Create a dictionary to store sequence records
    sequence_records = {}

    for strain_name in strain_names:
        # Define your query to search for the strain
        query = f"Coronavirus {strain_name} complete genome"

        # Use Entrez to search for the sequence and retrieve the first result
        handle = Entrez.esearch(db="nucleotide", term=query, retmax=1)
        record = Entrez.read(handle)
        handle.close()

        if record['Count'] == "0":
            print(f"No results found for {strain_name}.")
        else:
            # Retrieve the sequence record using the accession number
            accession = record['IdList'][0]

            # Introduce a delay to respect NCBI's rate limits
            time.sleep(3)

            handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
            seq_record = SeqIO.read(handle, "genbank")
            handle.close()

            # Convert the sequence to a string, pad it with 'N', and then convert it back to a Seq object
            seq_str = str(seq_record.seq)
            if len(seq_str) < common_length:
                padding = 'N' * (common_length - len(seq_str))
                seq_str += padding
            seq_record.seq = Seq(seq_str)

            sequence_records[strain_name] = seq_record

            # Save the sequence to a GenBank file
            sequence_file = f"{strain_name}_sequence.gb"
            SeqIO.write(seq_record, sequence_file, "genbank")
            print(f"Downloaded {strain_name} sequence and saved it to {sequence_file}")

    # Perform MSA for all sequences
    alignment = MultipleSeqAlignment(list(sequence_records.values()))
    output_file = "coronavirus_alignment.aln"

    # Save the MSA in ClustalW format
    SeqIO.write(alignment, output_file, "clustal")

    # Calculate percent identity and identify mutations
    seq_length = len(alignment[0])
    mutations = []

    for i in range(seq_length):
        column = alignment[:, i]
        column_sequences = [Seq(seq) for seq in column]
        column_letters = [str(seq) for seq in column_sequences]
        column_counter = Counter(column_letters)

        if len(column_counter) > 1:
            # There is a mutation at this position
            mutations.append((i + 1, column_counter))

    percent_identity = 100 * (1 - len(mutations) / seq_length)

    # Print mutations and percent identity
    print(f"Mutations:")
    for position, counts in mutations:
        print(f"Position {position}: {counts}")
    print(f"Percent Identity: {percent_identity:.2f}%")

    # Comment on the pathogenesis of the mutations (if any)
    # You can add your comments based on the mutations identified.

    return percent_identity

# Example of how to use the Q1 function with sample test case arguments
sample_test_case = ["Alpha", "Beta", "Gamma"]
common_length = 30287
output = Q1(sample_test_case, common_length)


Streaming output truncated to the last 5000 lines.
Position 1872: Counter({'G': 2, 'N': 1})
Position 1873: Counter({'T': 2, 'N': 1})
Position 1874: Counter({'A': 2, 'N': 1})
Position 1875: Counter({'C': 2, 'N': 1})
Position 1876: Counter({'C': 2, 'N': 1})
Position 1877: Counter({'A': 2, 'N': 1})
Position 1878: Counter({'A': 2, 'N': 1})
Position 1879: Counter({'C': 2, 'N': 1})
Position 1880: Counter({'C': 2, 'N': 1})
Position 1881: Counter({'T': 2, 'N': 1})
Position 1882: Counter({'C': 2, 'N': 1})
Position 1883: Counter({'A': 2, 'N': 1})
Position 1884: Counter({'C': 2, 'N': 1})
Position 1885: Counter({'C': 2, 'N': 1})
Position 1886: Counter({'A': 2, 'N': 1})
Position 1887: Counter({'A': 2, 'N': 1})
Position 1888: Counter({'C': 2, 'N': 1})
Position 1889: Counter({'A': 2, 'N': 1})
Position 1890: Counter({'T': 2, 'N': 1})
Position 1891: Counter({'T': 2, 'N': 1})
Position 1892: Counter({'A': 2, 'N': 1})
Position 1893: Counter({'C': 2, 'N': 1})
Position 1894: Counter({'A': 2, 'N': 1})
Positi

In [ ]:
#q2:
import networkx as nx
import matplotlib.pyplot as plt

def find_kmers(sequence, k):
    kmers = [sequence[i:i+k] for i in range(len(sequence)-k+1)]
    return kmers
def plot_de_bruijn_graph(de_bruijn_graph, output_filename):
    pos = nx.spring_layout(de_bruijn_graph)
    nx.draw(de_bruijn_graph, pos, with_labels=True)
    plt.savefig(output_filename)
    plt.show()
def assemble_sequence_from_graph(de_bruijn_graph):
    def find_eulerian_path(graph):
        start_node = next((node for node, neighbors in graph.items() if len(neighbors) % 2 != 0), next(iter(graph)))
        path, stack = [], [start_node]
        while stack:
            node = stack[-1]
            if graph[node]:
                stack.append(graph[node].pop())
            else:
                path.append(stack.pop())
        return path[::-1]

    graph = {node: list(neighbors) for node, neighbors in de_bruijn_graph.items()}
    for neighbors in de_bruijn_graph.values():
        for neighbor in neighbors:
            if neighbor not in graph:
                graph[neighbor] = []

    path = find_eulerian_path(graph)
    sequence = ''.join(node[-1] for node in path[1:])
    return path[0] + sequence

def build_de_bruijn_graph(kmers):
    graph = {}
    for kmer in kmers:
        prefix = kmer[:-1]
        suffix = kmer[1:]
        if prefix in graph:
            graph[prefix].append(suffix)
        else:
            graph[prefix] = [suffix]
    return graph

def Q2(test_case):
    sequence, k = test_case
    kmers = find_kmers(sequence, k)
    de_bruijn_graph = build_de_bruijn_graph(kmers)
    assembled_sequence = assemble_sequence_from_graph(de_bruijn_graph)
    return assembled_sequence


def menu():
    functions = [
        {"description": "Find K-mers and Assemble Sequence", "function": Q1},
        {"description": "Plot De Bruijn Graph", "function": plot_de_bruijn_graph}

    ]

    print("---------Welcome to my program-----------:")
    for i in range(len(functions)):
        print(str(i + 1) + ": " + functions[i]['description'])

    choice = int(input("Enter the number of the option you want to execute: ")) - 1
    if 0 <= choice < len(functions):
        return functions[choice]['function']
    else:
        print("Invalid choice. Exiting program.")
        return None
def build_networkx_de_bruijn_graph(de_bruijn_dict):
    G = nx.DiGraph()
    for node, neighbors in de_bruijn_dict.items():
        for neighbor in neighbors:
            G.add_edge(node, neighbor)
    return G

if __name__ == "__main__":
    function = menu()
    if function:
        test_case = ("TGTAGAAAGTACCCAGTGCTCAGTATAG", 5)
        if function == Q2:
            output = function(test_case)
            print("Assembled Sequence:", output)
        else:
            output_filename = "de_bruijn_graph.png"
            kmers = find_kmers(test_case[0], test_case[1])
            de_bruijn_dict = build_de_bruijn_graph(kmers)
            de_bruijn_graph = build_networkx_de_bruijn_graph(de_bruijn_dict)
            function(de_bruijn_graph, output_filename)
    else:
        print("Exiting program.")

---------Welcome to my program-----------:
1: Find K-mers and Assemble Sequence
2: Plot De Bruijn Graph


In [ ]:
#q3

def BWT(str_input):
  rotations=[]
  for i in range(len(str_input)):
    rotations.append(str_input[i:]+ str_input[:i])

  rotations.sort()
  bwt=""
  for rotation in rotations:
    bwt+=rotation[-1]


  return bwt

def Q3(test_case):
    transformed_input=BWT(test_case)
    return transformed_input


BWT_output=Q3("GCGTGCCTGGTCA$")
print(BWT_output)



ACTGGCT$TGCGGC
